In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import sys

# 1️⃣ Chuyển working directory tới root project (nơi có thư mục src)
%cd /content/drive/MyDrive/chatbot

# 2️⃣ Thêm folder root vào sys.path để Python tìm src
sys.path.append(os.getcwd())

# 3️⃣ Kiểm tra
!ls
# Nên thấy thư mục src, app_gradio.py, notebooks, v.v.


/content/drive/MyDrive/chatbot
app_gradio.ipynb  eval_result		  src
data		  model_evaluation.ipynb  train_models.ipynb
data_generating   models
data_viz.ipynb	  rag_evaluation.ipynb


In [3]:
!pip install langchain langchain-community langchain-google-genai google-generativeai transformers torch sentence-transformers pinecone gradio peft bitsandbytes accelerate pymongo --quiet
!pip install pinecone --upgrade --quiet
!pip install -U bitsandbytes --quiet
!pip install rank-bm25 unidecode -q
!pip install pymongo -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9

In [ ]:
import os
import gradio as gr

from src.logic_module import (
    list_sessions,
    create_session_handler,
    load_session_handler,
    delete_session_handler,
    chat_fn
)

# -------------------------- 1️⃣ Gradio UI --------------------------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 💬 Vietnam Legal Assistance")

    with gr.Row():
        # Sidebar: session management
        with gr.Column(scale=1):
            session_dropdown = gr.Dropdown(
                label="Select Session",
                choices=[lbl for lbl, _ in list_sessions()],
                interactive=True
            )
            new_name = gr.Textbox(label="New Session Name (optional)", placeholder="Enter new session name...")
            create_btn = gr.Button("➕ Create New")
            load_btn = gr.Button("📂 Load Session")
            delete_btn = gr.Button("🗑️ Delete Session")
            refresh_btn = gr.Button("🔄 Refresh List")

        # Main chat area
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=480)
            refs_output = gr.HTML()
            with gr.Row():
                msg = gr.Textbox(label="Question", placeholder="Type your question here...", lines=2)
                send = gr.Button("📨 Send")
            clear_btn = gr.Button("🧹 Clear Chat (local)")

    # -------------------------- 2️⃣ State --------------------------
    state_session = gr.State(value=None)
    state_cache = gr.State(value=[])  # retrieval cache

    # -------------------------- 3️⃣ Events --------------------------

    # Refresh session list
    refresh_btn.click(
        lambda: gr.update(choices=[lbl for lbl,_ in list_sessions()]),
        None,
        session_dropdown
    )

    # Create new session
    create_btn.click(
        create_session_handler,
        inputs=[new_name],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Load existing session
    load_btn.click(
        load_session_handler,
        inputs=[session_dropdown],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Delete session
    delete_btn.click(
        delete_session_handler,
        inputs=[session_dropdown],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Send chat message
    send.click(
        chat_fn,
        inputs=[state_session, chatbot, msg, state_cache],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )
    msg.submit(
        chat_fn,
        inputs=[state_session, chatbot, msg, state_cache],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Clear chat
    clear_btn.click(
        lambda: (None, [], [], ""),
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Auto-load session list when UI starts
    def load_session_list():
        return gr.update(choices=[lbl for lbl, _ in list_sessions()])

    demo.load(
        load_session_list,
        inputs=None,
        outputs=session_dropdown
    )

# -------------------------- 4️⃣ Launch --------------------------
if __name__ == "__main__":
    demo.launch(server_name="127.0.0.1", server_port=7861, debug=True)


/tmp/ipython-input-3269000665.py:13: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3269000665.py:32: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=480)
/tmp/ipython-input-3269000665.py:32: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=480)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bad5f8e360e67a086e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://bad5f8e360e67a086e.gradio.live


# Del Session

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.getenv("MONGO_URI")
db_name = os.getenv("MONGO_DB")
col_name = os.getenv("MONGO_COLLECTION")

from pymongo import MongoClient
result = MongoClient(uri)[db_name][col_name].delete_many({})
print("Đã xoá", result.deleted_count, "document.")


Đã xoá 35 document.
